<a href="https://colab.research.google.com/github/Equisoain/03MAIR---Algoritmos-de-Optimizacion/blob/main/Algoritmos_de_Optimizaci%C3%B3n_2a_conv.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Algoritmos de optimización - Trabajo Práctico<br>
Nombre y Apellidos: Maddi Equisoain Redin  <br>
Url: https://github.com/Equisoain/03MAIR---Algoritmos-de-Optimizacion/blob/main/Trabajo_Pr%C3%A1ctico_Algoritmos_de_Optimizaci%C3%B3n.ipynb <br>
Google Colab: https://colab.research.google.com/drive/1OsnrlHeTsOTwtVMitZ8ApzjMnvJz-rm8 <br>
Problema:

>**2. Organizar los horarios de partidos de una jornada de La Liga**

Descripción del problema:

Desde la La Liga de fútbol profesional se pretende organizar los horarios de los partidos de liga de cada jornada. Se conocen algunos datos que nos deben llevar a diseñar un algoritmo que realice la asignación de los partidos a los horarios de forma que maximice la audiencia.

Los horarios disponibles se conocen a priori y son los siguientes:


<table>
    <tr>
        <td>Viernes</td>
        <td>20</td>
    </tr>
    <tr>
        <td>Sábado</td>
        <td>12, 16, 18, 20</td>
    </tr>
    <tr>
        <td>Domingo</td>
        <td>12, 16, 18, 20</td>
    </tr>
    <tr>
        <td>Lunes</td>
        <td>20</td>
    </tr>
</table>

En primer lugar se clasifican los equipos en tres categorías según el numero de
seguidores( que tiene relación directa con la audiencia). Hay **3 equipos** en la **categoría A**, **11 equipos** de **categoría B** y **6 equipos** de **categoría C**.

Se conoce estadísticamente la audiencia que genera cada partido según los equipos que se enfrentan y en horario de sábado a las 20h (el mejor en todos los casos).

|  <!-- -->       | Categoría A | Categoría B  | Categoría C   |
|-----------------|-------------|--------------|---------------|
| **Categoría A** | 2 Millones  | 1.3 Millones | 1 Millones    |
| **Categoría B** |  <!-- -->   | 0.9 Millones | 0.75 Millones |
| **Categoría C** |  <!-- -->   | <!-- -->     | 0.47 Millones |

Si el horario del partido no se realiza a las 20 horas del sábado se sabe que se reduce según los coeficientes de la siguiente tabla.

Debemos asignar obligatoriamente siempre un partido el viernes y un partido el lunes.

| <!-- --> | Viernes | Sábado  | Domingo | Lunes |
|----------|---------|---------|---------|-------|
| **12h**  | -       | 0.55    | 0.45    | -     |
| **16h**  | -       | 0.7     | 0.75    | -     |
| **18h**  | -       | 0.8     | 0.85    | -     |
| **20h**  |  0.4    | 1       | 1       | 0.4   |

Es posible la coincidencia de horarios pero en este caso la audiencia de cada partido se verá afectada y se estima que se reduce en porcentaje según la siguiente tabla dependiendo del número de coincidencias:

| Coincidencias | _%  |
|---------------|-----|
| 0             | 0%  |
| 1             | 25% |
| 2             | 45% |
| 3             | 60% |
| 4             | 70% |
| 5             | 75% |
| 6             | 78% |
| 7             | 80% |
| 8             | 80% |

Los cálculos asociados a una jornada de ejemplo se realizan según se muestra en la siguiente tabla:

| Partido|Categorías|Horario|Base(Mill.)|Ponderación|Base*Ponderación|Corrección Coincidencia|
|----------------------|-----|-----|-----|-----|-----|-----|
|Celta - Real Madrid   | B-A | V20 | 1,3 | 0,4 | 0,52| 0,52|
|Valencia - R. Sociedad| B-A | S12 | 1,3 | 0,55| 0,72| 0,72|
|Mallorca - Eibar      | C-C | S16 | 0.47| 0,7 | 0,33| 0,33|
|Athletic - Barcelona  | B-A | S18 | 1,3 | 0,8 | 1,04| 1,04|
|Leganés - Osasuna     | C-C | S20 | 0,47| 1   | 0,47| 0,47|
|Villareal - Granada   | B-C | D16 | 0,75| 0,75| 0,56| 0,42|
|Alavés - Levante      | B-B | D16 | 0,9 | 0,75| 0,68| 0,51|
|Espanyol - Sevilla    | B-B | D18 | 0,9 | 0,85| 0,77| 0,77|
|Bétis - Valladolid    | B-C | D20 | 0,75| 1   | 0,75| 0,75|
|Atlético - Getafe     | B-B | L20 | 0,9 | 0,4 | 0,36| 0,36|

#Modelo
- ¿Como represento el espacio de soluciones?
- ¿Cual es la función objetivo?
- ¿Como implemento las restricciones?

- **¿Como represento el espacio de soluciones?**

El espacio de soluciones viene dado por el conjunto de partidos P (10 en este caso) y el conjunto de horarios H (10 franjas horarias) los cuales se agrupan como un diccionario:

`
{('Celta', 'Betis'): 'D18', ('Villarreal', 'Sevilla'): 'L20', ...},
`

donde las claves son los partidos que se juegan en la jornada y los valores el horario junto a la inicial del día. Por lo tanto, el espacio de soluciones es:

$$
\text{Espacio de soluciones} = H^P = 10^{10}.
$$


- **¿Cual es la función objetivo?**

En este problema la función objetivo es maximizar la audiencia total. Por lo tanto la función objetivo es:

$$ f_{objetivo} = max ∑_{i ∈ P} (A_i \cdot C_j(i) \cdot (1 - R(k_j(i)))),
 $$

donde, $A_i$ es la audiencia base del partido según a la categoría que pertenece cada equipo. $C_j$ es el coeficiente de penalización dependiendo la franja horaria. $R(k_j)$ es la penalización de audiencia debido a coincidencias en el mismo horario.

- **¿Como implemento las restricciones?**

Estas son las restricciones que se deben imponer al modelo:

1. Reducción por horario del partido si no se juega a las 20h.

| <!-- --> | Viernes | Sábado  | Domingo | Lunes |
|----------|---------|---------|---------|-------|
| **12h**  | -       | 0.55    | 0.45    | -     |
| **16h**  | -       | 0.7     | 0.75    | -     |
| **18h**  | -       | 0.8     | 0.85    | -     |
| **20h**  |  0.4    | 1       | 1       | 0.4   |

$$ Audiencia_i = A_i \cdot C(h_i),$$

se puede observar la restricción en la función objetivo descrita anteriormente donde $A_i$ es la audiecia base y $C_j$ es el coeficiente de penalización dependiendo la franja horaria.

2. Se ha de jugar un partido el viernes y otro el lunes:

$$\sum_{i\in P} x_{i,V20} = 1, \sum_{i\in P} x_{i,L20} = 1.$$

3. Reducción de audiencia por coincidencia:

| Coincidencias ($k_j$) | _%  | $R(k_j)$  |
|-----------------------|-----|-----------|
| 0                     | 0%  | 0.0       |
| 1                     | 25% | 0.25      |
| 2                     | 45% | 0.45      |
| 3                     | 60% | 0.60      |
| 4                     | 70% | 0.70  |
| 5                     | 75% | 0.75  |
| 6                     | 78% | 0.78 |
| 7                     | 80% | 0.8  |
| 8                     | 80% | 0.8  |

$$
\text{Audiencia reducida por coincidencia}_i = A_i \cdot C(h_i) \cdot (1 - R(k_{h_i})),
$$

donde $R(k)$ es la penalicación de audiencia por coincidencia de horario.



#Análisis
- **¿Que complejidad tiene el problema?. Orden de complejidad y Contabilizar el espacio de soluciones**


Tenemos **20 equipos** en la liga y cada jornada cuenta con **10 partidos**. Cada equipo solo puede jugar un partido por jornada. Contamos con **10** opciones de **horarios** donde pueden haber partidos:

`
['V20', 'S12', 'S16', 'S18', 'S20', 'D12', 'D16', 'D18', 'D20', 'L20']
`

Por lo tanto, el espacio de soluciones, sin tener en cuenta las restricciones, es:
 $$
\text{Espacio de soluciones} = 10^{10}
 $$

Esto da 10 mil millones d ecombinaciones posibles, lo que hace que la búsqueda por fuerza bruta no sea viable. Teniendo en cuenta las restricciones el número aumenta. La **complejidad** del problema es **exponencial** $O(m^n)$, clasificándose como **NP-difícil**. Por ello, es conveniente utilizar algoritmos metaheurísticos.


#Diseño
- **¿Que técnica utilizo? ¿Por qué?**

Al tratarse de un problema de combinatoria la resolución por fuerza bruta se vuelve inviable. Por ello, podremos resolverlo mediante métodos voraces vistos en clase como divide y vencerás y vuelta atrás o con algoritmos metaherísticos como son los algoritmos genéticos.

Entre los diferentes métodos propuestos divide y vencerás se dificulta a la hora de dividir el problema principal en subpreblemas independientes. El método vuelta atrás al imponer tantas restricciones el coste computacional se eleva. Por ello, nos hemos decantado por **algoritmos genéticos** debido a que se aplican muchas condiciones y restricciones complejas. Este método encuentra buenas soluciones explorando múltiples soluciones simultáneamente utilizando mutaciones y cruce para mejorar los resultados. Además, tiene menor coste computacional que los métodos mencionados anteriormente.

In [11]:
import random
import pandas as pd

###### DEFINIMOS LAS VARIABLES DADAS ######
horarios = ['V20', 'S12', 'S16', 'S18', 'S20', 'D12', 'D16', 'D18', 'D20', 'L20']

partidos = [
    ('Celta', 'Real Madrid'), ('Valencia', 'R. Sociedad'), ('Mallorca', 'Eibar'),
    ('Athletic', 'Barcelona'), ('Leganés', 'Osasuna'), ('Villarreal', 'Granada'),
    ('Alavés', 'Levante'), ('Espanyol', 'Sevilla'), ('Betis', 'Valladolid'), ('Atlético', 'Getafe')
]

categorias_equipos = {
    'Real Madrid': 'A', 'Barcelona': 'A', 'Atlético': 'A', 'Sevilla': 'A',
    'R. Sociedad': 'B', 'Betis': 'B', 'Villarreal': 'B', 'Athletic': 'B',
    'Valencia': 'B', 'Celta': 'B', 'Osasuna': 'C', 'Mallorca': 'C',
    'Espanyol': 'C', 'Leganés': 'C', 'Eibar': 'C', 'Getafe': 'C',
    'Granada': 'C', 'Levante': 'C', 'Alavés': 'C', 'Valladolid': 'C'
}

audiencia_base = {
    ('A', 'A'): 2.0, ('A', 'B'): 1.3, ('A', 'C'): 1.0,
    ('B', 'A'): 1.3, ('B', 'B'): 0.9, ('B', 'C'): 0.75,
    ('C', 'A'): 1.0, ('C', 'B'): 0.75, ('C', 'C'): 0.47
}

penalizacion_horario = {
    'V20': 0.4, 'S12': 0.55, 'S16': 0.7, 'S18': 0.8, 'S20': 1,
    'D12': 0.45, 'D16': 0.75, 'D18': 0.85, 'D20': 1, 'L20': 0.4
}

penalizacion_coincidencia = [0, 0.25, 0.45, 0.60, 0.70, 0.75, 0.78, 0.80, 0.80]

def generar_tabla(asignacion):
    tabla = []
    horarios_usados = {h: 0 for h in horarios}

    for partido, horario in asignacion.items():
        cat_local = categorias_equipos[partido[0]]
        cat_visitante = categorias_equipos[partido[1]]
        categoria_partido = f"{cat_local}-{cat_visitante}"

        audiencia_base_partido = audiencia_base[(cat_local, cat_visitante)]
        ponderacion = round(penalizacion_horario[horario], 2)
        base_ponderada = round(audiencia_base_partido * ponderacion, 2)
        horarios_usados[horario] += 1

        tabla.append([f"{partido[0]} - {partido[1]}", categoria_partido, horario, audiencia_base_partido, ponderacion, base_ponderada])

    for fila in tabla:
        coincidencias = horarios_usados[fila[2]] - 1
        correccion = fila[5] * (1 - penalizacion_coincidencia[coincidencias]) if coincidencias > 0 else fila[5]
        fila.append(correccion)

    return tabla

def algoritmo_genetico(poblacion_size=100, generaciones=500):
    poblacion = []
    for _ in range(poblacion_size):
        asignacion = {}

        partido_viernes = partidos[0]
        partido_lunes = partidos[1]
        asignacion[partido_viernes] = 'V20'
        asignacion[partido_lunes] = 'L20'

        partidos_restantes = [p for p in partidos if p not in (partido_viernes, partido_lunes)]
        horarios_restantes = [h for h in horarios if h not in ('V20', 'L20')]
        random.shuffle(horarios_restantes)

        for i, partido in enumerate(partidos_restantes):
            asignacion[partido] = horarios_restantes[i % len(horarios_restantes)]

        poblacion.append(asignacion)

    for _ in range(generaciones):
        poblacion = sorted(poblacion, key=lambda x: sum(row[-1] for row in generar_tabla(x)), reverse=True)
        nueva_poblacion = poblacion[:10]

        while len(nueva_poblacion) < poblacion_size:
            p1, p2 = random.sample(poblacion[:50], 2)
            hijo = {}

            hijo[partido_viernes] = 'V20'
            hijo[partido_lunes] = 'L20'
            for partido in partidos_restantes:
                hijo[partido] = random.choice([p1[partido], p2[partido]])

            nueva_poblacion.append(hijo)

        poblacion = nueva_poblacion

    mejor_solucion = max(poblacion, key=lambda x: sum(row[-1] for row in generar_tabla(x)))
    return generar_tabla(mejor_solucion)


tabla_resultado = algoritmo_genetico()

df_resultado = pd.DataFrame(tabla_resultado, columns=[
    "Partido", "Categoría", "Horario", "Base (Mill.)", "Ponderación", "Base * Ponderación", "Corrección Coincidencia"
])

#print(df_resultado.to_markdown(index=False))
print(df_resultado.to_string(index=False))


               Partido Categoría Horario  Base (Mill.)  Ponderación  Base * Ponderación  Corrección Coincidencia
   Celta - Real Madrid       B-A     V20          1.30         0.40                0.52                     0.52
Valencia - R. Sociedad       B-B     L20          0.90         0.40                0.36                     0.36
      Mallorca - Eibar       C-C     D12          0.47         0.45                0.21                     0.21
  Athletic - Barcelona       B-A     D20          1.30         1.00                1.30                     1.30
     Leganés - Osasuna       C-C     S12          0.47         0.55                0.26                     0.26
  Villarreal - Granada       B-C     D16          0.75         0.75                0.56                     0.56
      Alavés - Levante       C-C     S16          0.47         0.70                0.33                     0.33
    Espanyol - Sevilla       C-A     D18          1.00         0.85                0.85         